In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import ydata_profiling

%matplotlib inline

In [2]:
# Load the dataset
df = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\ccfraudTrain.xlsx')

df1 = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\ccfraudTest.xlsx')
# For snakey diagram
#df1 = pd.read_excel(r'C:\Users\sg_cl\Desktop\masters subjects\DSRT 736\Fraud Dataset\New_data_CCFD\Sankey_data.xlsx')mk

In [ ]:
df.columns

In [ ]:
print("Missing values:", df.isnull().sum())

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# checking the dataset
df.info()
df.describe()
df1.info()
df1.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   Unnamed: 0             1048575 non-null  int64         
 1   trans_date_trans_time  1048575 non-null  datetime64[ns]
 2   cc_num                 1048575 non-null  int64         
 3   merchant               1048575 non-null  object        
 4   category               1048575 non-null  object        
 5   amt                    1048575 non-null  float64       
 6   first                  1048575 non-null  object        
 7   last                   1048575 non-null  object        
 8   gender                 1048575 non-null  object        
 9   street                 1048575 non-null  object        
 10  city                   1048575 non-null  object        
 11  state                  1048575 non-null  object        
 12  zip                    10485

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,555719.000000,5.557190e+05,555719.000000,555719.000000,555719.000000,555719.000000,5.557190e+05,5.557190e+05,555719.000000,555719.000000,555719.000000
mean,277859.000000,4.178387e+17,69.392810,48842.628015,38.543253,-90.231325,8.822189e+04,1.380679e+09,38.542798,-90.231380,0.003860
std,160422.401459,1.309837e+18,156.745941,26855.283328,5.061336,13.721780,3.003909e+05,5.201104e+06,5.095829,13.733071,0.062008
min,0.000000,6.041621e+10,1.000000,1257.000000,20.027100,-165.672300,2.300000e+01,1.371817e+09,19.027422,-166.671575,0.000000
25%,138929.500000,1.800429e+14,9.630000,26292.000000,34.668900,-96.798000,7.410000e+02,1.376029e+09,34.755301,-96.905129,0.000000
50%,277859.000000,3.521417e+15,47.290000,48174.000000,39.371600,-87.476900,2.408000e+03,1.380762e+09,39.376593,-87.445204,0.000000
75%,416788.500000,4.635331e+15,83.010000,72011.000000,41.894800,-80.175200,1.968500e+04,1.385867e+09,41.954163,-80.264637,0.000000
max,555718.000000,4.992346e+18,22768.110000,99921.000000,65.689900,-67.950300,2.906700e+06,1.388534e+09,66.679297,-66.952026,1.000000


In [4]:
#print (df['trans_date_trans_time'].head())
from datetime import datetime as dt
import pytz  # Import the pytz module for time zones

df['trans_date_trans_time_HR'] = df['trans_date_trans_time'].dt.strftime("%H")

# Convert the Unix time values to datetime objects
df['unix_time_NEW'] = df['unix_time'].apply(lambda x: dt.fromtimestamp(x))

# Set the time zone to the desired time zone (e.g., 'UTC' or your local time zone)
local_time_zone = 'America/New_York'
df['unix_time_NEW'] = df['unix_time_NEW'].dt.tz_localize(pytz.utc).dt.tz_convert(local_time_zone)


print (df.head())

df1['trans_date_trans_time_HR'] = df1['trans_date_trans_time'].dt.strftime("%H")

# Convert the Unix time values to datetime objects
df1['unix_time_NEW'] = df1['unix_time'].apply(lambda x: dt.fromtimestamp(x))

# Set the time zone to the desired time zone (e.g., 'UTC' or your local time zone)
local_time_zone = 'America/New_York'
df1['unix_time_NEW'] = df1['unix_time_NEW'].dt.tz_localize(pytz.utc).dt.tz_convert(local_time_zone)


print (df1.head())

   Unnamed: 0 trans_date_trans_time            cc_num  \
0           0   2019-01-01 00:00:18  2703186189652090   
1           1   2019-01-01 00:00:44      630423337322   
2           2   2019-01-01 00:00:51    38859492057661   
3           3   2019-01-01 00:01:16  3534093764340240   
4           4   2019-01-01 00:03:06   375534208663984   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street            city state    zip  \
0    Banks      F                561 Perry Cove  Moravian Falls    NC  28654   
1     Gill      F  43039 Riley Greens Suite 39

In [5]:
df = df.drop('unix_time', axis=1)
df = df.drop('unix_time_NEW', axis=1)
df1 = df1.drop('unix_time', axis=1)
df1 = df1.drop('unix_time_NEW', axis=1)
#df = df.drop('Errors?', axis=1)
#df = df.drop('Time', axis=1)


In [ ]:
print (df.head())

In [6]:
print(df.corr())
df.columns

            Unnamed: 0    cc_num       amt       zip       lat      long  \
Unnamed: 0    1.000000  0.000575 -0.001397  0.001207 -0.000781 -0.001047   
cc_num        0.000575  1.000000  0.000760  0.041448 -0.058910 -0.047945   
amt          -0.001397  0.000760  1.000000  0.001567 -0.001916  0.000151   
zip           0.001207  0.041448  0.001567  1.000000 -0.113957 -0.909957   
lat          -0.000781 -0.058910 -0.001916 -0.113957  1.000000 -0.016041   
long         -0.001047 -0.047945  0.000151 -0.909957 -0.016041  1.000000   
city_pop     -0.000856 -0.008993  0.006485  0.079164 -0.156069 -0.053443   
merch_lat    -0.000690 -0.058611 -0.001832 -0.113250  0.993598 -0.015967   
merch_long   -0.001045 -0.047925  0.000156 -0.909147 -0.016033  0.999120   
is_fraud     -0.009041 -0.002301  0.218417 -0.001844  0.001355  0.002030   

            city_pop  merch_lat  merch_long  is_fraud  
Unnamed: 0 -0.000856  -0.000690   -0.001045 -0.009041  
cc_num     -0.008993  -0.058611   -0.047925 -0.0023

Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'merch_lat',
       'merch_long', 'is_fraud', 'trans_date_trans_time_HR'],
      dtype='object')

In [ ]:
profile = pandas_profiling.ProfileReport(df)
df['trans_date_trans_time_HR'].unique()
profile.to_file("data_profile_report.html")

In [ ]:
profile.to_file("data_profile_report.html")

In [ ]:
df.columns

In [7]:
# Sample dataset 1
# Convert 'Timestamp' column to datetime type
#main_df['Timestamp'] = pd.to_datetime(main_df['Timestamp'])

# Select the desired columns and perform aggregation on the subset
subset_df = df[[ 'cc_num', 'trans_date_trans_time_HR', 'merchant', 'category', 'gender','state','lat', 'long','merch_lat',
       'merch_long', 'amt','is_fraud']].reset_index()
subset_df1 = df1[[ 'cc_num', 'trans_date_trans_time_HR', 'merchant', 'category', 'gender','state','lat', 'long','merch_lat',
       'merch_long', 'amt','is_fraud']].reset_index()

In [ ]:
subset_df.head()

In [8]:
# Convert categorical variables to numerical using one-hot encoding
subset_df = pd.get_dummies(subset_df, columns=['merchant', 'category', 'gender', 'state'])

subset_df1 = pd.get_dummies(subset_df1, columns=['merchant', 'category', 'gender', 'state'])

In [9]:
#subset_df.head()
subset_df = subset_df.drop('state_DE', axis=1)
#subset_df1.head()
#subset_df1['state_DE'] = 0

In [ ]:
#Data pre processing String values to Int values
print(df['Is Fraud?'].unique())
y = df['Is Fraud?']
enc = preprocessing.LabelEncoder()
enc.fit(y)
y = enc.transform(y)
print(y)


# Separate features (X) and labels (y)
X = df.drop('Is Fraud?', axis=1)
print(X.head())
print(y)

In [10]:
# Split data into features (X) and target (y)
X = subset_df.drop('is_fraud', axis=1)
y = subset_df['is_fraud']

X1 = subset_df1.drop('is_fraud', axis=1)
y1 = subset_df1['is_fraud']

In [ ]:
print("Missing values:", df.isnull().sum())

In [11]:
# Split the dataset into training, validation, and test sets
X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.2, stratify=y, random_state=42)
#X_val,X_test,y_val, y_test = train_test_split(X1, y1,test_size=0.5, stratify=y1, random_state=42)
#X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

In [12]:
# Print the shapes of the resulting subsets
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Training set shape: (838860, 767) (838860,)
Test set shape: (209715, 767) (209715,)


In [13]:
# Create a linear regression model
# = LinearRegression()
# Define class weights (adjust these based on your problem)
genuine_weight = 1.0 #6006 / (1042569 + 6006)
fraud_weight = 9.0 #1042569 / (1042569 + 6006)

class_weights = {0: genuine_weight, 1: fraud_weight}
#class_weights = {0: 1.0, 1: 10.0}


model = RandomForestClassifier(n_estimators=100,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model1 = RandomForestClassifier(n_estimators=30,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model2 = RandomForestClassifier(n_estimators=70,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model3 = RandomForestClassifier(n_estimators=80,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)
model4 = RandomForestClassifier(n_estimators=90,class_weight=class_weights, random_state=42,n_jobs=-1,max_samples=0.8)

In [31]:
X_train.columns

Index(['index', 'cc_num', 'trans_date_trans_time_HR', 'lat', 'long',
       'merch_lat', 'merch_long', 'amt', 'merchant_fraud_Abbott-Rogahn',
       'merchant_fraud_Abbott-Steuber',
       ...
       'state_SD', 'state_TN', 'state_TX', 'state_UT', 'state_VA', 'state_VT',
       'state_WA', 'state_WI', 'state_WV', 'state_WY'],
      dtype='object', length=767)

In [32]:
cols_to_exclude = 'is_fraud'
X_train = subset_df.drop(columns = [cols_to_exclude])
y_train = subset_df[['is_fraud']]
X_test = subset_df1.drop(columns = [cols_to_exclude])
y_test = subset_df1[['is_fraud']]

In [33]:
# Train the model
model.fit(X_train, y_train)

# Train the model
model1.fit(X_train, y_train)
# Train the model
model2.fit(X_train, y_train)


model3.fit(X_train, y_train)

model4.fit(X_train, y_train)

C:\Users\sg_cl\AppData\Local\Temp\ipykernel_13436\1300458953.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
C:\Users\sg_cl\AppData\Local\Temp\ipykernel_13436\1300458953.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model1.fit(X_train, y_train)
C:\Users\sg_cl\AppData\Local\Temp\ipykernel_13436\1300458953.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model2.fit(X_train, y_train)
C:\Users\sg_cl\AppData\Local\Temp\ipykernel_13436\1300458953.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3.fit(X_trai

RandomForestClassifier(class_weight={0: 1.0, 1: 9.0}, max_samples=0.8,
                       n_estimators=90, n_jobs=-1, random_state=42)

In [34]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Make predictions on the test set
y1_pred = model1.predict(X_test)

# Make predictions on the test set
y2_pred = model2.predict(X_test)

# Make predictions on the test set
y3_pred = model3.predict(X_test)

# Make predictions on the test set
y4_pred = model4.predict(X_test)

In [35]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y1_pred))
print("\nClassification Report:\n", classification_report(y_test, y1_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y2_pred))
print("\nClassification Report:\n", classification_report(y_test, y2_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y3_pred))
print("\nClassification Report:\n", classification_report(y_test, y3_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y4_pred))
print("\nClassification Report:\n", classification_report(y_test, y4_pred))

Mean Squared Error: 0.002060393832134586
Confusion Matrix:
 [[553529     45]
 [  1100   1045]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.96      0.49      0.65      2145

    accuracy                           1.00    555719
   macro avg       0.98      0.74      0.82    555719
weighted avg       1.00      1.00      1.00    555719

Confusion Matrix:
 [[553521     53]
 [  1132   1013]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.95      0.47      0.63      2145

    accuracy                           1.00    555719
   macro avg       0.97      0.74      0.81    555719
weighted avg       1.00      1.00      1.00    555719

Confusion Matrix:
 [[553524     50]
 [  1109   1036]]

Classification Report:
               precision    recall  f1-score   support

           0 

In [36]:
 # Replace clf with your Random Forest model and X_test with your test data
predicted_probabilities = model.predict_proba(X_test)[:, 1] 
predicted_probabilities1 = model1.predict_proba(X_test)[:, 1] 
predicted_probabilities2 = model2.predict_proba(X_test)[:, 1] 
predicted_probabilities3 = model3.predict_proba(X_test)[:, 1] 
predicted_probabilities4 = model4.predict_proba(X_test)[:, 1] 

In [ ]:
predicted_probabilities4

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y_pred = (predicted_probabilities >= threshold).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y1_pred = (predicted_probabilities1 >= threshold).astype(int)
    precision = precision_score(y_test, y1_pred)
    recall = recall_score(y_test, y1_pred)
    f1 = f1_score(y_test, y1_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y2_pred = (predicted_probabilities2 >= threshold).astype(int)
    precision = precision_score(y_test, y2_pred)
    recall = recall_score(y_test, y2_pred)
    f1 = f1_score(y_test, y2_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [ ]:
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]  # Example threshold values
for threshold in thresholds:
    y3_pred = (predicted_probabilities3 >= threshold).astype(int)
    precision = precision_score(y_test, y3_pred)
    recall = recall_score(y_test, y3_pred)
    f1 = f1_score(y_test, y3_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")

In [39]:
thresholds = [0.2] 
#thresholds = [0.2,0.3, 0.4, 0.5, 0.6, 0.7,0.8]  # Example threshold values
for threshold in thresholds:
    y4_pred = (predicted_probabilities4 >= threshold).astype(int)
    precision = precision_score(y_test, y4_pred)
    recall = recall_score(y_test, y4_pred)
    f1 = f1_score(y_test, y4_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y4_pred))
    print("\nClassification Report:\n", classification_report(y_test, y4_pred))

Threshold: 0.20 | Precision: 0.71 | Recall: 0.74 | F1-score: 0.72
Confusion Matrix:
 [[552922    652]
 [   555   1590]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.71      0.74      0.72      2145

    accuracy                           1.00    555719
   macro avg       0.85      0.87      0.86    555719
weighted avg       1.00      1.00      1.00    555719



In [20]:
# Split the dataset into training, validation, and test sets
#X_train,X_test, y_train,y_test = train_test_split(X, y,test_size=0.2, stratify=y, random_state=42)
X_val,X_test,y_val, y_test = train_test_split(X1, y1,test_size=0.5, stratify=y1, random_state=42)

In [21]:
# Make predictions on the test set
y4_pred = model4.predict(X_test)

In [23]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(838860, 767)
(277860, 767)
(838860,)
(277860,)


In [24]:
thresholds = [0.3] 
#thresholds = [0.2,0.3, 0.4, 0.5, 0.6, 0.7,0.8]  # Example threshold values
for threshold in thresholds:
    y4_pred = (predicted_probabilities4 >= threshold).astype(int)
    precision = precision_score(y_test, y4_pred)
    recall = recall_score(y_test, y4_pred)
    f1 = f1_score(y_test, y4_pred)
    print(f"Threshold: {threshold:.2f} | Precision: {precision:.2f} | Recall: {recall:.2f} | F1-score: {f1:.2f}")
    print("Confusion Matrix:\n", confusion_matrix(y_test, y4_pred))
    print("\nClassification Report:\n", classification_report(y_test, y4_pred))

ValueError: Found input variables with inconsistent numbers of samples: [277860, 209715]

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, thresholds = precision_recall_curve(y_test, predicted_probabilities4)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

In [ ]:
# Add the predicted labels to the dataset
X_test['predicted_label'] = y_pred

In [ ]:
y.columns


In [ ]:
category_totals = df.groupby('is_fraud')['is_fraud'].count()
print(category_totals)

In [ ]:
X_test.columns

In [ ]:
# Identify false negative transactions
false_negatives = X_test[(X_test['predicted_label'] == 0) & (X_test['is_fraud'] == 1)]
print(false_negatives)

In [ ]:
print(y_pred)
# Set a threshold
threshold = 0.01

# Convert predictions to binary classes based on the threshold
y_pred_binary = [1 if val >= threshold else 0 for val in y_pred]

# Print the binary predictions
print("Binary Predictions:", y_pred_binary)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)

In [ ]:
# Select an example for validation (optional)
example_index = 208
example_features = X.iloc[example_index]
example_target = y[example_index]

# Make predictions on the example
example_prediction = model.predict([example_features])

# Compare with actual value
print("Example features:", example_features)
print("Example target:", example_target)
print("Example prediction:", example_prediction)

# Calculate mean squared error on the test set (optional)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
print(y_pred)

In [ ]:
counts = y_pred_binary.value_counts()